In [2]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint
import pandas as pd
import numpy as np
import csv

In [3]:
pip install boto3


Note: you may need to restart the kernel to use updated packages.


In [4]:
def load_json(bank_name):
    mydict={"absa":'ABSA.json',
           "fnb":"FNB.json",
            "bidvest":"Bidvest.json",
            "standardbank":"Standard Bank.json",
            "nedbank":"Nedbank.json"
           }
    with open(mydict[bank_name]) as json_file: 
        data = json.load(json_file)
    
    return data[0]['Blocks']

In [5]:
def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '    
    return text

In [6]:
def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                        
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows

In [7]:
def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)
    
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():
        
        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'
        
    csv += '\n\n\n'
    return csv


In [8]:
def generate_bad_code(bank_name):
    blocks=load_json(bank_name)
    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
         "<b> NO Table FOUND </b>"
            

    return table_blocks,blocks_map

In [9]:
def create_csv(bank_name):  
    '''
    creates a csv and given a bank name ;bank name is the name of the bank beng used by thr company 
    '''
    csv = ''
    table_blocks,blocks_map=generate_bad_code(bank_name)
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv.strip("\n")

In [10]:
csv=create_csv("absa")

In [11]:
csv

'Table: Table_1\n\nDate ,Transaction Description ,Charge ,,Debit Amount ,Credit Amount ,Balance ,\n5/03/2019 5/03/2019 ,Bal Brought Forward Airtime Debit Tiq001absa Card No. 4971 Mtn: 0730403965 ,1.50 ,T ,499.00 ,,4 794.48 4 295.48 ,\n7/03/2019 ,Acb Debit:External Settlement lairdressihairdressing V148 ,19.00 ,T ,3 662.00 ,,633.48 ,\n12/03/2019 ,Acb Credit Settlement Hollard Etimage004-0016 ,,,,6 298.80 ,6 932.28 ,\n15/03/2019 ,Acb Debit:Externa Settlement Centrafin Debit 033424:029008 ,19.00 ,T ,1 325.16 ,,5 607.12 ,\n18/03/2019 ,Airtime Debit Tiq001absa ,1.50 ,T ,110.00 ,,5 497.12 ,\n19/03/2019 ,Card No. 4971 Vodacom: 0824941097 bank Payment To Settlement Absa Bank Wire It Upstairs ,,,1 000.00 ,,4 497.12 ,\n27/03/2019 ,Npf Credit Treas/lbs ,,,,10 000.00 ,14 497.12 ,\n27/03/2019 ,Eftbbl2tllvpf002/Images 1 Fnb bank Transfer Settlement ,,,13 000.00 ,,1 497.12 ,\n29/03/2019 ,1400154585 Absa Bank 1400154585 Acb Debit:External Settlement ,19.00 ,T ,591.82 ,,905.30 ,\n,Nashua Ec 7127 Nashua

In [18]:
def generate_info(csv,bank_name='Absa'):
    
    if bank_name.lower()=='absa': 
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.find(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_2\n\n","")
        
       
        
    elif bank_name.lower()=='fnb':
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.rfind(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_3\n\n","") 
        
    elif bank_name.lower()=='bidvest':
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.find(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_2\n\n","")
  
    elif bank_name.lower()=='nedbank':
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.find(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_2\n\n","")
          
    elif bank_name.lower()=='standardbank':
        bankstatement=csv[:csv.find(6*"\n")]
        bankstatement=bankstatement.replace("Table: Table_1\n\n","")
        banksummary=csv[csv.find(6*"\n")+6:]
        banksummary=banksummary.replace("Table: Table_2\n\n","")
        
    
    return bankstatement,banksummary
        
    
    
  




In [19]:
def writecsv(csv,output_table1,output_table2):

    """" Write bank statement and bank summart csv """
    bankstatement_tuple=generate_info(csv)
    with open(output_table1, "wt") as bank:
        bank.write(str(bankstatement_tuple[0]))
    with open(output_table2, "wt") as bank:
        bank.write(str(bankstatement_tuple[1]))
    
        

In [20]:
writecsv(csv,"bank_statement.csv","bank_summary.csv")

In [21]:
bank_df=pd.read_csv("bank_statement.csv")

In [22]:
summary_df=pd.read_csv("bank_summary.csv").reset_index()

In [23]:
print(summary_df)

                      index  Account Summary:  Unnamed: 1 Unnamed: 2  \
0  Balance Brought Forward                 NaN      4 794        48    
1           Sundry Credits                 NaN     51 298        80    
2            Sundry Debits                 NaN     52 416       70-    
3                  Charges                 NaN      1 920       50-    
4                  Balance                 NaN      1 756        08    
5          Overdraft Limit                 NaN          0        00    

   Unnamed: 3  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
5         NaN  


In [24]:
print(summary_df.columns)

Index(['index', 'Account Summary: ', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'], dtype='object')


In [25]:
summary_df=summary_df[['index', 'Unnamed: 1', 'Unnamed: 2',]]

In [26]:
summary_df.columns=['Account Summary', 'Rands', 'Cents']

In [27]:
summary_df

,Account Summary,Rands,Cents
0,Balance Brought Forward,4 794,48
1,Sundry Credits,51 298,80
2,Sundry Debits,52 416,70-
3,Charges,1 920,50-
4,Balance,1 756,08
5,Overdraft Limit,0,00


In [28]:
amount=lambda x,y:x+'.'+y

In [29]:
summary_df["Amount"]=amount(summary_df["Rands"],summary_df["Cents"])

In [30]:
summary_df

,Account Summary,Rands,Cents,Amount
0,Balance Brought Forward,4 794,48,4 794.48
1,Sundry Credits,51 298,80,51 298.80
2,Sundry Debits,52 416,70-,52 416.70-
3,Charges,1 920,50-,1 920.50-
4,Balance,1 756,08,1 756.08
5,Overdraft Limit,0,00,0.00


In [31]:
summary_df.drop(["Rands","Cents"],axis=1,inplace=True)

In [32]:
summary_df

,Account Summary,Amount
0,Balance Brought Forward,4 794.48
1,Sundry Credits,51 298.80
2,Sundry Debits,52 416.70-
3,Charges,1 920.50-
4,Balance,1 756.08
5,Overdraft Limit,0.00


In [33]:
Sign= lambda X:'-' if X.strip().endswith('-') else "+"

In [34]:
summary_df["sign"]=summary_df["Amount"].apply(Sign)

In [35]:
import ast

In [36]:
summary_df["Amount"]=summary_df["Amount"].apply(lambda X:float(X.replace(" ","").replace("-","")))

In [37]:
summary_df

,Account Summary,Amount,sign
0,Balance Brought Forward,4794.48,+
1,Sundry Credits,51298.80,+
2,Sundry Debits,52416.70,-
3,Charges,1920.50,-
4,Balance,1756.08,+
5,Overdraft Limit,0.00,+


In [38]:
bank_df.head()

,Date,Transaction Description,Charge,Unnamed: 3,Debit Amount,Credit Amount,Balance,Unnamed: 7
0,5/03/2019 5/03/2019,Bal Brought Forward Airtime Debit Tiq001absa C...,1.5,T,499.00,NaN,4 794.48 4 295.48,NaN
1,7/03/2019,Acb Debit:External Settlement lairdressihairdr...,19.0,T,3 662.00,NaN,633.48,NaN
2,12/03/2019,Acb Credit Settlement Hollard Etimage004-0016,NaN,NaN,NaN,6 298.80,6 932.28,NaN
3,15/03/2019,Acb Debit:Externa Settlement Centrafin Debit 0...,19.0,T,1 325.16,NaN,5 607.12,NaN
4,18/03/2019,Airtime Debit Tiq001absa,1.5,T,110.00,NaN,5 497.12,NaN


In [39]:
bank_df.tail()

,Date,Transaction Description,Charge,Unnamed: 3,Debit Amount,Credit Amount,Balance,Unnamed: 7
13,NaN,1400154585 Absa Bank 1400154585,NaN,NaN,NaN,NaN,NaN,NaN
14,30/03/2019,Internet Bank Fee Settlement,NaN,*k,1 052.50,NaN,9 852.80,NaN
15,1/04/2019,Monthly Acc Fee Headoffice,NaN,*k,75.00,NaN,9 777.80,NaN
16,1/04/2019,Transaction Charge Headoffice,NaN,*k,193.00,NaN,9 584.80,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
summary_df.head().T

,0,1,2,3,4
Account Summary,Balance Brought Forward,Sundry Credits,Sundry Debits,Charges,Balance
Amount,4794.48,51298.8,52416.7,1920.5,1756.08
sign,+,+,-,-,+


In [41]:
df_statement=pd.read_json('ABSA.json')

In [42]:
df_statement['Blocks']

0    [{'BlockType': 'PAGE', 'Geometry': {'BoundingB...
1    [{'BlockType': 'WORD', 'Confidence': 99.950119...
Name: Blocks, dtype: object

In [43]:
output_file = 'output.csv'

# replace content
 with open(output_file, "wt") as fout:
  fout.write(str(csv))

   fout.close()


IndentationError: unexpected indent (<ipython-input-43-41288abbb103>, line 4)

In [44]:
statement_df= pd.read_csv('bank_statement.csv', delimiter=',')

In [45]:
statement_df.replace({np.NaN:0},inplace=True)

In [46]:
statement_df

,Date,Transaction Description,Charge,Unnamed: 3,Debit Amount,Credit Amount,Balance,Unnamed: 7
0,5/03/2019 5/03/2019,Bal Brought Forward Airtime Debit Tiq001absa C...,1.5,T,499.00,0,4 794.48 4 295.48,0.0
1,7/03/2019,Acb Debit:External Settlement lairdressihairdr...,19.0,T,3 662.00,0,633.48,0.0
2,12/03/2019,Acb Credit Settlement Hollard Etimage004-0016,0.0,0,0,6 298.80,6 932.28,0.0
3,15/03/2019,Acb Debit:Externa Settlement Centrafin Debit 0...,19.0,T,1 325.16,0,5 607.12,0.0
4,18/03/2019,Airtime Debit Tiq001absa,1.5,T,110.00,0,5 497.12,0.0
5,19/03/2019,Card No. 4971 Vodacom: 0824941097 bank Payment...,0.0,0,1 000.00,0,4 497.12,0.0
6,27/03/2019,Npf Credit Treas/lbs,0.0,0,0,10 000.00,14 497.12,0.0
7,27/03/2019,Eftbbl2tllvpf002/Images 1 Fnb bank Transfer Se...,0.0,0,13 000.00,0,1 497.12,0.0
8,29/03/2019,1400154585 Absa Bank 1400154585 Acb Debit:Exte...,19.0,T,591.82,0,905.30,0.0
9,0,Nashua Ec 7127 Nashua Ec,0.0,0,0,0,0,0.0


In [47]:
statement_df=statement_df[['Date ', 'Transaction Description ', 'Charge ',
       'Debit Amount ', 'Credit Amount ', 'Balance ']]

In [48]:
statement_df=statement_df[statement_df['Date ']!= 0]

In [49]:
statement_df

,Date,Transaction Description,Charge,Debit Amount,Credit Amount,Balance
0,5/03/2019 5/03/2019,Bal Brought Forward Airtime Debit Tiq001absa C...,1.5,499.00,0,4 794.48 4 295.48
1,7/03/2019,Acb Debit:External Settlement lairdressihairdr...,19.0,3 662.00,0,633.48
2,12/03/2019,Acb Credit Settlement Hollard Etimage004-0016,0.0,0,6 298.80,6 932.28
3,15/03/2019,Acb Debit:Externa Settlement Centrafin Debit 0...,19.0,1 325.16,0,5 607.12
4,18/03/2019,Airtime Debit Tiq001absa,1.5,110.00,0,5 497.12
5,19/03/2019,Card No. 4971 Vodacom: 0824941097 bank Payment...,0.0,1 000.00,0,4 497.12
6,27/03/2019,Npf Credit Treas/lbs,0.0,0,10 000.00,14 497.12
7,27/03/2019,Eftbbl2tllvpf002/Images 1 Fnb bank Transfer Se...,0.0,13 000.00,0,1 497.12
8,29/03/2019,1400154585 Absa Bank 1400154585 Acb Debit:Exte...,19.0,591.82,0,905.30
10,30/03/2019,Npf Credit Treas/lbs,0.0,0,15 000.00,15 905.30
